# Analyse reindexation from ABES - April 2023, corrected in June 2023

# Set project

### Packages

In [1]:
# Import librairies
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings



from utils_metrics import *
from utils_visualization import *

2023-06-29 11:11:48.941762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package words to /home/aurelie/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aurelie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aurelie/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

### Graphical parameters

In [3]:
# Parametres graphiques
%matplotlib inline
rcParams = {
    'font.size': 14,
    'font.family':'sans-serif',
    'font.sans-serif':['Liberation Sans'],
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.max_open_warning': 30}

sns.set(rc=rcParams)
sns.set_style(
    "whitegrid", {
        'axes.edgecolor': 'k',
        'axes.linewidth': 1,
        'axes.grid': True,
        'xtick.major.width': 1,
        'ytick.major.width': 1
        })

sns.set_context(
    "notebook",
    font_scale=1.1,
    rc={"lines.linewidth": 1.5})

pd.set_option('display.max_columns', None)

### Paths

In [4]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "/data"
data_path_reindex = "/home/aurelie/ABES/labo-indexation-ai/data/reindexation/correction_juin2023"
data_path_embed = "/home/aurelie/ABES/labo-indexation-ai/data/embeddings/juin2023"
output_path = path + "/outputs"
fig_path = path + "/figs"

In [5]:
# Include embeddings
include_embedding = True

# Import data

In [6]:
# Original dataset
reindex = "fusion_fichiers_reindexation.xlsx"
rameau_file = "rameau_Tf_Td.csv"
embedding = "embeddings_results_valid100.csv"

### Check reindexation files

In [7]:
# Merge with already merged file for concept correction
df = pd.read_excel(os.path.join(data_path_reindex, reindex), index_col=0)
print(df.shape)
df.head()

(112, 16)


,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD
0,79.0,000308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'État ""fort"" à la...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,NaN,NaN,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,NaN,NaN
1,58.0,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,NaN,NaN,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,NaN,NaN
2,59.0,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,NaN,NaN,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,NaN,NaN
3,65.0,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences,606 $3027248283$aPhilosophie des sciences,Philosophie des sciences,NaN,NaN,606$3027248283$aPhilosophie des sciences\n606$...,Philosophie des sciences;;Futur (philosophie),606$3027582957$aLoi (philosophie)\n606$3027365...,Loi (philosophie);;Philosophie et sciences,NaN,NaN
4,80.0,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,606 $3027234312$aImpérialisme$3027365581$xPouv...,Impérialisme -- Pouvoir (sciences sociales) --...,NaN,NaN,606$3027365581$aPouvoir (sciences sociales)$30...,Pouvoir (sciences sociales) -- Histoire -- Rom...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie);;Empereurs -- ...,NaN,NaN


### Reorganise data

In [8]:
def import_indexation(file):
    df = pd.read_excel(
        os.path.join(data_path_reindex, file), 
        sheet_name="Notices à indexer", 
        engine="openpyxl", 
        converters={"PPN": lambda x: str(x)})
    df.drop(df[df["PPN"].isna()].index, inplace=True) # delete first line corresponding to example
    df["PPN"] = df["PPN"].astype(str)
    
    return df

In [9]:
# Set list of indexeurs
suffix_list = ["AFE", "MCR", "JMF", "LPL", "LJZ", "MPD"]

In [10]:
# Raw initial import 
index_file = '100 NOTICES A INDEXER_AFE.xlsx'
data = import_indexation(index_file)
print(data.shape)
data

(100, 6)


,N°ordre,PPN,TITRE,RESUME,INDEXATION UNIMARC,INDEXATION AFFICHAGE PUBLIC
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire\nPou...
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France‎ -- 1958-.... (5e Répu...
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,606$3027873757$aCulte impérial$302724301X$yRom...,Culte impérial -- Rome (Italie)\nEmpereurs -- ...
...,...,...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ...",606$3050573004$aDroit public$3027226794$yFranc...,Droit public --France -- Histoire\nEtat -- Res...
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...,606$3027280721$aArtistes$3027622444$ySuède$308...,Artistes -- Suède -- 20e siècle
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ...",606$3027267547$aRévoltes$3027226794$yFrance$30...,Révoltes -- France -- 2000-….\nRésistance poli...
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...,606$3183790669$aMusique et géographie\n606$302...,Musique et géographie\nRécits de voyages -- Da...


In [11]:
# Remove useless columns
data.drop(columns=["INDEXATION UNIMARC", "INDEXATION AFFICHAGE PUBLIC"], inplace=True)
data

,N°ordre,PPN,TITRE,RESUME
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ..."
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ..."
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon..."
3,65,058296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod..."
4,80,059911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...
...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ..."
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ..."
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...


In [12]:
# Recreate indexation files
index_columns = []
for suf in suffix_list:
    print(f"Merging {suf} reindexation")
    unimarc = str("INDEX_UNI_" + suf)
    public = str("INDEX_" + suf)
    temp = df[["PPN", unimarc, public]].dropna()
    data = pd.merge(data, temp, on="PPN", how="outer")
    print("dimension after merge: ", data.shape)
    index_columns.append(public)

Merging AFE reindexation
dimension after merge:  (100, 6)
Merging MCR reindexation
dimension after merge:  (100, 8)
Merging JMF reindexation
dimension after merge:  (100, 10)
Merging LPL reindexation
dimension after merge:  (100, 12)
Merging LJZ reindexation
dimension after merge:  (100, 14)
Merging MPD reindexation
dimension after merge:  (100, 16)


In [13]:
data.head(3)

,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,606$3027223345$aClasses dirigeantes$302779430X...,Classes dirigeantes -- Relations avec l'État -...,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Éli...
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,606$3028696980$aIntellectuels français$3049647...,Intellectuels français -- Sociologie ;;Intelle...,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,606 ##‎$3027235149Intellectuels‎$3027284948$yF...,Intellectuels -- France -- 1958-.... (5e Répub...
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,606$3027431215$aPhilosophie bouddhique \n606$3...,Philosophie bouddhique ;;Spiritualité bouddhique,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,606$3027321835$aBouddhisme,Bouddhisme


In [14]:
#Save final reindexation file
data.to_csv(os.path.join(data_path, "reindexation_final_juin2023.csv"))

## Check files

In [15]:
# Import rameau list
ram = pd.read_csv(os.path.join(data_path, rameau_file), index_col=0)
print("Nombre de notices différentes: ", ram.shape[0])
ram.head(10)

Nombre de notices différentes:  103021


,NOM
0,Pica (oiseaux)
1,Langues brittoniques
2,Consentement (droit canonique)
3,Diplomates écrivains
4,Augustines
5,Accélérateurs cosmiques
6,Et le Québec
7,Propagande allemande
8,Navigateurs (logiciels)
9,Lymphangiectasie


In [16]:
# Embeddings
emb = pd.read_csv(os.path.join(data_path_embed, embedding), index_col=0)
emb["PPN"] = emb["PPN"].astype(str)
emb["embedding_chains"] = emb["embedding_chains"].apply(lambda x: eval(x))
emb["embeddings_autorites"] = emb["embeddings_autorites"].apply(lambda x: eval(x))
print("Dimension of the dataset: ", emb.shape)
emb.head()

Dimension of the dataset:  (100, 3)


,PPN,embedding_chains,embeddings_autorites
0,000308838,"[État, Administration publique, Fonctionnaires...","[État, Administration publique, Fonctionnaires..."
1,00094758X,"[Dollar américain, Système monétaire internati...","[Dollar américain, Système monétaire internati..."
2,003632806,"[Intellectuels -- Activité politique, Intellec...","[Intellectuels, Activité politique, Intellectu..."
3,047450037,"[Bouddhisme -- Anthologies, Bouddhisme -- Livr...","[Bouddhisme, Anthologies, Bouddhisme, Livres s..."
4,05224170X,"[Professeurs de sciences de l'éducation, Appre...","[Professeurs de sciences de l'éducation, Appre..."


In [17]:
if include_embedding:
    data = data.merge(emb, on="PPN", how="inner")
    print("Dimension after merge with embeddings: ", data.shape)
    # Check merging
    data.head(5)
    # Save merged file
    data.to_csv(os.path.join(data_path, "data_with_reindexation_and_embeddings.csv"))

Dimension after merge with embeddings:  (100, 18)


## Check all concepts

In [34]:
ram

,NOM
0,Pica (oiseaux)
1,Langues brittoniques
2,Consentement (droit canonique)
3,Diplomates écrivains
4,Augustines
...,...
103016,"Accordéon et guitare, Musique d'"
103017,Christianisme et franc-maçonnerie
103018,"Cithare, Musique de (cithares (2))"
103019,Prothèses internes


In [35]:
# Mettre à plat TOUS les mots clé
pattern = r';;\s*(?![^()$]*\))| -- |-- | --|--| - '

for col in index_columns:
    name = col.split("_")[1]
    print("Dealing with: ", name)
    new_col = str('rameau_concept_' + name)
    data[new_col] = data[col].apply(lambda x: re.split(pattern, str(x)))


Dealing with:  AFE
Dealing with:  MCR
Dealing with:  JMF
Dealing with:  LPL
Dealing with:  LJZ
Dealing with:  MPD


In [36]:
data.head(3)

,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,INDEX_UNI_LPL,INDEX_LPL,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD,embedding_chains,embeddings_autorites,rameau_concept_AFE,rameau_concept_MCR,rameau_concept_JMF,rameau_concept_LPL,rameau_concept_LJZ,rameau_concept_MPD
0,79,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,606$3027223345$aClasses dirigeantes$302779430X...,Classes dirigeantes -- Relations avec l'État -...,606$3027322610$aHauts-fonctionnairesss$3027226...,Hauts-fonctionnairesss -- France;;Classes diri...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts-fonctionn...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Éli...,"[État, Administration publique, Fonctionnaires...","[État, Administration publique, Fonctionnaires...","[Classes dirigeantes, France, Histoire, Pouvoi...","[Pouvoir (sciences sociales), Classes dirigean...","[Classes dirigeantes, Relations avec l'État, F...","[Hauts-fonctionnairesss, France, Classes dirig...","[Classes dirigeantes, France, Hauts-fonctionna...","[Classes dirigeantes, France, Histoire, Élite ..."
1,58,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,606$3028696980$aIntellectuels français$3049647...,Intellectuels français -- Sociologie ;;Intelle...,606$027235149$aIntellectuels$3027284948$yFranc...,Intellectuels -- France -- 1958 (5e République...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,606 ##‎$3027235149Intellectuels‎$3027284948$yF...,Intellectuels -- France -- 1958-.... (5e Répub...,"[Intellectuels -- Activité politique, Intellec...","[Intellectuels, Activité politique, Intellectu...","[Intellectuels, France, 1958-.... (5e Républiq...","[Intellectuels, France, 1945]","[Intellectuels français, Sociologie , Intellec...","[Intellectuels, France, 1958 (5e République), ...","[Intellectuels, France, Vie intellectuelle, Fr...","[Intellectuels, France, 1958-.... (5e Républiq..."
2,59,047450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,606$3027431215$aPhilosophie bouddhique \n606$3...,Philosophie bouddhique ;;Spiritualité bouddhique,606$3027321835$aBouddhisme$3027789853$xEtude e...,Bouddhisme -- Étude et enseignement;;Philosoph...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,606$3027321835$aBouddhisme,Bouddhisme,"[Bouddhisme -- Anthologies, Bouddhisme -- Livr...","[Bouddhisme, Anthologies, Bouddhisme, Livres s...",[Bouddhisme],[Bouddhisme],"[Philosophie bouddhique , Spiritualité bouddhi...","[Bouddhisme, Étude et enseignement, Philosophi...","[Bouddhisme, Coutumes et pratiques, Bouddhisme...",[Bouddhisme]


## Format data

In [37]:
df.columns

Index(['N°ordre', 'PPN', 'TITRE', 'RESUME', 'INDEX_UNI_AFE', 'INDEX_AFE',
       'INDEX_UNI_MCR', 'INDEX_MCR', 'INDEX_UNI_JMF', 'INDEX_JMF',
       'INDEX_UNI_LPL', 'INDEX_LPL', 'INDEX_UNI_LJZ', 'INDEX_LJZ',
       'INDEX_UNI_MPD', 'INDEX_MPD'],
      dtype='object')

In [38]:
col_to_keep = ["PPN", "rameau_concept_AFE",
               "rameau_concept_MCR", "rameau_concept_JMF",
                "rameau_concept_LPL", "rameau_concept_LJZ", "rameau_concept_MPD", "embeddings_autorites"]
df_pivot = pd.melt(data[col_to_keep], id_vars=["PPN"])
df_pivot

,PPN,variable,value
0,000308838,rameau_concept_AFE,"[Classes dirigeantes, France, Histoire, Pouvoi..."
1,003632806,rameau_concept_AFE,"[Intellectuels, France, 1958-.... (5e Républiq..."
2,047450037,rameau_concept_AFE,[Bouddhisme]
3,058296182,rameau_concept_AFE,[Philosophie et sciences]
4,059911174,rameau_concept_AFE,"[Culte impérial, Rome (Italie), Empereurs, Rom..."
...,...,...,...
695,23690454X,embeddings_autorites,"[Droit public, Droit, Histoire, Droit, Histoir..."
696,24155859X,embeddings_autorites,"[Art brut, Art, Expositions artistiques, Artis..."
697,25561280X,embeddings_autorites,"[Morale, Modernité, Idéologie, Individu et soc..."
698,26117309X,embeddings_autorites,"[Chansons françaises, Thèmes, motifs, Sexisme..."


In [39]:
labels=df_pivot["value"]
labels

0      [Classes dirigeantes, France, Histoire, Pouvoi...
1      [Intellectuels, France, 1958-.... (5e Républiq...
2                                           [Bouddhisme]
3                              [Philosophie et sciences]
4      [Culte impérial, Rome (Italie), Empereurs, Rom...
                             ...                        
695    [Droit public, Droit, Histoire, Droit, Histoir...
696    [Art brut, Art, Expositions artistiques, Artis...
697    [Morale, Modernité, Idéologie, Individu et soc...
698    [Chansons françaises, Thèmes,  motifs, Sexisme...
699    [Algorithmes, Algorithmes, Droit, Algorithmes,...
Name: value, Length: 700, dtype: object

In [41]:
def flatten(list):
    flat_list = [item for sublist in list for item in sublist]
    return flat_list

In [42]:
keywords = flatten(labels)
print(f"Le dataset contient {len(set(keywords))} concepts RAMEAU differents")

Le dataset contient 1245 concepts RAMEAU differents


In [43]:
all_concepts = data["rameau_concept_AFE"] + data["rameau_concept_MCR"] + data["rameau_concept_JMF"] + data["rameau_concept_LPL"] + data["rameau_concept_LJZ"]	+ data["rameau_concept_MPD"] + data["embeddings_autorites"]
keywords2 = flatten(all_concepts)
print(f"Le dataset contient {len(set(keywords2))} concepts RAMEAU differents")

Le dataset contient 1245 concepts RAMEAU differents


In [44]:
# Verification que toutes les vedettes appartiennent au referenciel RAMEAU
RAMEAU = ram["NOM"].to_list()
keywords = list(map(lambda x: x.lstrip().rstrip(), keywords)) 
if all([item in RAMEAU for item in keywords]):
    print("Tous les mots clés sont des concepts rameau")
else:
    check = [item in RAMEAU for item in keywords]
    bad_keywords = []
    for i in range(len(check)):
        if not check[i]:
            bad_keywords.append(keywords[i])
 
    unknown_concepts = list(set(bad_keywords))
    print(f"{len(unknown_concepts)} mots clés absents du répertoire RAMEAU sur un total de {len(set(keywords))}")

154 mots clés absents du répertoire RAMEAU sur un total de 1184


In [45]:
# Save list of badly formatted concepts, for check
with open(os.path.join(data_path, "concepts_a_verifier_reindexation_juin2023.txt"), "w") as output:
    output.write(str(unknown_concepts))
unknown_concepts

['Méthodes expérimentales',
 'Valence-en-Brie (Seine et Marne)',
 'Le Lazzaro (Calvados ; site archéologique)',
 'Île-de-France (France)',
 'Relations gouvernement central',
 '30 av. J.-C.-284',
 'Cancers',
 'Construction et conception',
 'Histoire et fiction',
 'Besoin (philosophie)',
 "Travaux d'aiguille",
 'Valence-en-Brie (Seine-et-Marne)',
 'Chez les enfants',
 'Vie soutenable',
 'Extrême-droite',
 'Colonialisme et ethnologie',
 'Nouvelle de science-fiction',
 'Performances (sport)',
 '1990-2020',
 'Mouvement anti-nucléaire',
 'Géants du web',
 'Art et Sciences',
 'Nucléaire',
 'Thèmes',
 '20e siècle',
 'Egypte',
 'Écrivains africains de la langue française',
 '1800-….',
 'Rome (Italie)',
 '2000-....',
 'Art, vente aux enchères',
 'Rome (Italie',
 'Pratiques sexuelles',
 '1945-2000',
 'Concept',
 'Suède',
 'Tadmur (Syrie ; région)',
 'Aspect socio-culturel',
 'Alimentation et nutrition',
 '1800-…',
 '30 av. J.-C-284',
 'Aspect cognitif',
 'Doctrines',
 '1945-….',
 'Méthodes pédago